In [3]:
import pandas as pd
steamdb = pd.read_csv('inputs/chart.csv')


In [4]:
display(steamdb.info())
steamdb.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 802 entries, 0 to 801
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DateTime  802 non-null    object 
 1   Players   802 non-null    int64  
 2   Flags     0 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 18.9+ KB


None

,DateTime,Players,Flags
0,2019-11-14 00:00:00,21797,NaN
1,2019-11-15 00:00:00,20594,NaN
2,2019-11-16 00:00:00,26163,NaN
3,2019-11-17 00:00:00,26767,NaN
4,2019-11-18 00:00:00,19596,NaN


# What info do we have?
- number of players per day since release of age of empires ii: definitive edition
- limited to steam players

# What could we do with this?
- compare against other games in the age series if we get the charts for those. this could help us understand the popularity of age
- compare against other top games or other RTS games (e.g., starcraft? but that's not on steam)